#### This code, developed by IIT Ropar students Vrushank Ahire, Nikhil Pakhale, Murru Sai Yaswanth, and Guna Challa, utilizes novel algorithms/data structures/techniques.

In [ ]:
# In .env file
GROQ_API_KEY= "insert your Api Key"
COHERE_API_KEY= "insert your Api Key"

In [ ]:
pip install -qU langchain

In [ ]:
pip install -q pydub

In [ ]:
import os
import base64
import datetime
import whisper
import pandas as pd
from io import BytesIO
from pydub import AudioSegment
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain.memory import ChatMessageHistory
from IPython.display import HTML, display
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:


# Set up API keys
os.environ['GROQ_API_KEY'] = GROQ_API_KEY  # Replace with your actual key
os.environ['COHERE_API_KEY'] = COHERE_API_KEY  # Replace with your actual key
load_dotenv()
# Create test document
with open('/content/customer_conversation.txt', 'w') as f:
    f.write("""
    Welcome to our customer support system.
    We're here to help with your questions and concerns.
    Our services include technical support, billing assistance, and general inquiries.
    """)

class CustomerSupportBot:
    def __init__(self, instruction_document_path):
        """Initialize the customer support bot with necessary models and components."""
        # Initialize Whisper model with weights_only=True
        self.whisper_model = whisper.load_model("base", device="cpu")

        # Verify API keys
        if not os.getenv('GROQ_API_KEY'):
            raise ValueError("GROQ_API_KEY is not set")
        if not os.getenv('COHERE_API_KEY'):
            raise ValueError("COHERE_API_KEY is not set")

        # Initialize Groq Chat model
        self.chat = ChatGroq(
            temperature=0.7,
            model="Use your Model",
            api_key=os.getenv('GROQ_API_KEY')
        )

        self.history = ChatMessageHistory()
        self.conversation_log = []
        self._setup_rag(instruction_document_path)

        # Register callbacks
        from google.colab import output
        output.register_callback('process_audio', self.process_audio_callback)
        output.register_callback('end_session', self.end_session_callback)

    def _setup_rag(self, document_path):
        """Setup RAG components with the instruction document."""
        try:
            with open(document_path) as f:
                document_text = f.read()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=150,
                chunk_overlap=20
            )
            texts = text_splitter.create_documents([document_text])

            embeddings = CohereEmbeddings(
                model="embed-english-v2.0",
                cohere_api_key=os.getenv('COHERE_API_KEY')
            )
            self.db = FAISS.from_documents(texts, embeddings)
            self.retriever = self.db.as_retriever()
        except Exception as e:
            print(f"Error setting up RAG: {str(e)}")
            raise

    def transcribe_audio(self, audio_data):
        """Transcribe audio data to text using Whisper."""
        try:
            audio = AudioSegment.from_file(
                BytesIO(base64.b64decode(audio_data)),
                format="webm"
            )
            audio.export("current_audio.wav", format="wav")

            result = self.whisper_model.transcribe("current_audio.wav")
            return result["text"]
        except Exception as e:
            print(f"Error transcribing audio: {str(e)}")
            raise

    def generate_response(self, query):
        """Generate response using RAG and Groq chat model."""
        try:
            docs = self.retriever.get_relevant_documents(query)
            context = "\n".join([doc.page_content for doc in docs])

            messages = [
                SystemMessage(content="You are a helpful customer support agent. Use the context provided to answer questions accurately and professionally."),
                HumanMessage(content=f"Context: {context}\n\nUser Query: {query}")
            ]

            response = self.chat(messages)

            self.history.add_user_message(query)
            self.history.add_ai_message(response.content)

            return response.content
        except Exception as e:
            print(f"Error generating response: {str(e)}")
            raise

    def analyze_interaction(self, text):
        """Analyze interaction for churn risk and technical assistance needs."""
        churn_keywords = ["cancel", "end service", "not satisfied", "too expensive"]
        tech_keywords = ["help", "support", "issue", "problem", "assistance"]

        churn_risk = any(keyword in text.lower() for keyword in churn_keywords)
        technical_assistance = any(keyword in text.lower() for keyword in tech_keywords)

        return churn_risk, technical_assistance

    def log_interaction(self, query, response):
        """Log the interaction with analysis."""
        churn_risk, technical_assistance = self.analyze_interaction(query)

        interaction = {
            "timestamp": datetime.datetime.now().isoformat(),
            "query": query,
            "response": response,
            "churn_risk": churn_risk,
            "technical_assistance": technical_assistance
        }
        self.conversation_log.append(interaction)
        return interaction

    def save_log(self, filename="conversation_log.csv"):
        """Save conversation log to CSV."""
        try:
            df = pd.DataFrame(self.conversation_log)
            df.to_csv(filename, index=False)
            print(f"Conversation log saved to {filename}")
        except Exception as e:
            print(f"Error saving log: {str(e)}")

    def process_audio_callback(self, audio_data):
        """Callback for processing audio data."""
        try:
            transcription = self.transcribe_audio(audio_data)
            print(f"User Query: {transcription}")

            response = self.generate_response(transcription)
            self.log_interaction(transcription, response)
            print(f"Agent Response: {response}")
        except Exception as e:
            print(f"Error in audio processing callback: {str(e)}")

    def end_session_callback(self):
        """Callback for ending the session."""
        try:
            self.save_log()
            print("Session ended. Log saved.")
        except Exception as e:
            print(f"Error in end session callback: {str(e)}")

    def display_interface(self):
        """Display the recording interface."""
        return HTML("""
            <div style="padding: 20px; border: 1px solid #ccc; border-radius: 5px;">
                <h3>Customer Support Recording Interface</h3>
                <button id="startRecording"
                        style="background-color: #4CAF50; color: white; padding: 10px 20px; margin: 5px; border: none; border-radius: 4px; cursor: pointer;"
                        onclick="startRecording()">
                    Start Recording
                </button>
                <button id="stopRecording"
                        style="background-color: #f44336; color: white; padding: 10px 20px; margin: 5px; border: none; border-radius: 4px; cursor: pointer;"
                        onclick="stopRecording()">
                    Stop Recording
                </button>
                <button id="endSession"
                        style="background-color: #2196F3; color: white; padding: 10px 20px; margin: 5px; border: none; border-radius: 4px; cursor: pointer;"
                        onclick="endSession()">
                    End Session
                </button>
                <div id="status" style="margin-top: 10px;"></div>
            </div>
            <script>
                var mediaRecorder;
                var audioChunks = [];

                async function startRecording() {
                    try {
                        document.getElementById('status').innerHTML = 'Starting recording...';
                        let stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                        mediaRecorder = new MediaRecorder(stream);
                        mediaRecorder.start();
                        audioChunks = [];
                        mediaRecorder.ondataavailable = event => audioChunks.push(event.data);
                        document.getElementById('status').innerHTML = 'Recording...';
                    } catch (error) {
                        document.getElementById('status').innerHTML = 'Error starting recording: ' + error;
                    }
                }

                async function stopRecording() {
                    try {
                        document.getElementById('status').innerHTML = 'Stopping recording...';
                        mediaRecorder.stop();
                        mediaRecorder.onstop = async () => {
                            const audioBlob = new Blob(audioChunks, { type: 'audio/webm' });
                            const reader = new FileReader();
                            reader.readAsDataURL(audioBlob);
                            reader.onloadend = () => {
                                google.colab.kernel.invokeFunction(
                                    'process_audio',
                                    [reader.result.split(',')[1]],
                                    {}
                                );
                            };
                            document.getElementById('status').innerHTML = 'Processing audio...';
                        };
                    } catch (error) {
                        document.getElementById('status').innerHTML = 'Error stopping recording: ' + error;
                    }
                }

                function endSession() {
                    document.getElementById('status').innerHTML = 'Ending session...';
                    google.colab.kernel.invokeFunction('end_session', [], {});
                }
            </script>
        """)

def setup_bot():
    """Setup and return the bot interface."""
    try:
        instruction_path = '/content/customer_support.txt'
        if not os.path.exists(instruction_path):
            raise FileNotFoundError(f"Instruction document not found at {instruction_path}")

        bot = CustomerSupportBot(instruction_path)
        return bot.display_interface()
    except Exception as e:
        print(f"Error setting up bot: {str(e)}")
        raise

In [ ]:
# Usage
# Run this to display the interface
interface = setup_bot()
display(interface)